In [1]:
import tensorflow as tf
import face_detection
import dlib
from fer import FER
import cv2
import os
from mtcnn import MTCNN
import keras
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import numpy as np
import torch
import gc
from datetime import datetime
import time

%matplotlib inline

Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.


In [2]:
print(tf.__version__)
print(tf.test.is_gpu_available())

2.3.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


31-07-2020:13:58:14,148 WARNING  [deprecation.py:323] From <ipython-input-2-2e82a26757ae>:2: is_gpu_available (from tensorflow.python.framework.test_util) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False


In [10]:
# traing path 

test_flag = 1

if test_flag == 1:
    run_type = "test"
else:
    run_type = "train"

cur_path = os.path.abspath(os.getcwd())
img_dir = os.path.join(cur_path, run_type+"\\img")
print(img_dir)
result_dir = os.path.join(cur_path, run_type+"\\result")

img_list = os.listdir(img_dir) 
print(img_list[:10])

C:\Users\parkh\Desktop\[NOTA]facial_emotion_data\test\img
['101.1042343192.jpg', '105.crop2.JPG', '108.inline_image_preview.jpg', '111.crop6.JPG', '112.crop3.JPG', '113.crop.JPG', '115.187551_smile-girl-hd-wallpaper.jpg', '115.Tingey-AsianTour-e1562308309822.jpg', '118.c9f63d1b4291fff3a21a2a64dc9b649564ecc61bv2_hq.jpg', '118.crop.JPG']


In [15]:
# name = "2.jpg"
# img = cv2.imread(name)
# detector = FER()
# detect_res = detector.detect_emotions(img)

# print(detect_res)
# position = detect_res[0]['box']
# print(detect_res[0]['box'])


def run(start, end):  
    
    # emotion detector
    emotion_detector = FER()
    
    cnt = 0
    for img_name in img_list[start:end]:  
            
        # img
        img_path = os.path.join(img_dir, img_name)

        # BGR to RGB
#         image = cv2.imread(img_path)[:, :, ::-1]
        image = cv2.imread(img_path)
        
        # face position
        detections = emotion_detector.detect_emotions(image)

        # show origin
        print("<",img_name, ">")
#         plt.imshow(image,aspect="auto")
#         plt.show()
        if len(detections) == 0 :
            cv2.imwrite(result_dir+"\\failed\\"+img_name,image)  
#             print("detecting failed !!")
        else:
#             print("detecting sucessed !!")
            pass

        for i in range(len(detections)):
            margin = 0
#             print(detections[0]['box'])
            
#             detections['box']
            top = int(round(detections[i]['box'][1]))-margin
            top = max(0, top)
            bottom = top + int(round(detections[i]['box'][3]))+margin
            left = int(round(detections[i]['box'][0]))-margin
            left = max(0,left)
            right =  left + int(round(detections[i]['box'][2]))+margin

            crop_image = image[top:bottom, left:right]
#             plt.imshow(crop_image,aspect="auto")
#             plt.show()
            
    #         print(crop_image)
    #         plt.imshow(crop_image,aspect="auto")
    #         plt.show()
    #         p = detections

            # show window
#             win = dlib.image_window()
#             win.set_image(crop_image)

            # emotion 
            
            
#             print(detections)
            save_name = img_name[:-4] +"_"+ str(i) +".jpg"
#             print(crop_image)
#             print(detector.top_emotion(crop_image))           
            top_emotion, score = emotion_detector.top_emotion(image)

            if top_emotion == 'fear':
                top_emotion = 'surprise'
            if top_emotion == 'diguste':
                top_emotion = 'anger'

            cv2.imwrite(result_dir+"\\"+top_emotion+"\\"+save_name,crop_image)
#             print("emotion is ", top_emotion)

           
#             del top_emotion
                

            
            del crop_image               
        del detections
        del image
        del img_path
        
    del emotion_detector
    
    


In [16]:
batch_size = 300
epoch = 0
start = 0
end = len(img_list)

start_time = time.time()
print("batch size: ", batch_size, "total epoch:", end-start)
for i in range(start,end,batch_size):
    epoch += batch_size
    print("\nepoch: ", epoch)
    torch.cuda.empty_cache()
    gc.collect()
    run(i,i+batch_size)
    
end_time = time.time()
print("elapsed time: ", end_time-start_time)


batch size:  300 total epoch: 184

epoch:  300
< 101.1042343192.jpg >
< 105.crop2.JPG >
< 108.inline_image_preview.jpg >
< 111.crop6.JPG >
< 112.crop3.JPG >
< 113.crop.JPG >
< 115.187551_smile-girl-hd-wallpaper.jpg >
< 115.Tingey-AsianTour-e1562308309822.jpg >
< 118.c9f63d1b4291fff3a21a2a64dc9b649564ecc61bv2_hq.jpg >
< 118.crop.JPG >
< 120.crop3.JPG >
< 123.jbox1k9.jpg >
< 128.crop.JPG >
< 131.crop2.JPG >
< 131.photo-23-group-photo.jpg >
< 139.cms_temp_article_16170744788284.jpg >
< 14.smiling-asian-woman-happy-selfie.leungchopan.Shutterstock-370x242.jpg >
< 142.6116-07084961en_Masterfile.jpg >
< 147.201810172009181710_1.jpg >
< 147.AAG1.jpg >
< 158.100010110-1536813813150christine_web_2018.jpg >
< 167.121477348-a-surprised-asian-person.jpg >
< 168.crop.JPG >
< 170.article_21153839568579.jpg >
< 175.https%3A%2F%2Fs3-ap-northeast-1.amazonaws.com%2Fpsh-ex-ftnikkei-3937bb4%2Fimages%2F0%2F6%2F7%2F2%2F19232760-2-eng-GB%2FCropped-1549438550AP_18205777720895.jpg >
< 177.crop3.JPG >
< 182.maxr

In [86]:
end_time = time.time()
print("elapsed time: ", end_time-start_time)

elapsed time:  401.26791405677795


In [ ]:
emotion, score = detector.top_emotion(img)
print(emotion, score)